In [59]:
import numpy as np
import matplotlib.pyplot as plt

Reading Andriej's article and coding along.


In [60]:

# RNN API: 
# just a simple step()

class RNN():
  def __init__(self, ):
    pass
  def step(self, x):
    # update the hidden state
    self.h = np.tanh(np.dot(self.W_hh, self.h) + np.dot(self.W_xh, x))

    # compute the output vec
    y = np.dot(self.W_hy, self.h)
    return y



What are the W_hh, W_xh, W_hy, etc?
What is self.h?

W_* are matrices.
self.h is a vector.

Two terms inside tanh
one based on previous hidden state and one based on current input.
In each step we matrix-multiply both previous hidden state and current input with some matrices (parameters), add them together, and squash the result with tanh into [-1, 1] range.

Matrices are initialized randomly.
hidden state at first is zero.






In [61]:
import tinygrad
from tinygrad import Tensor


In [62]:
tinygrad.Device._devices

['DSP',
 'LLVM',
 'PYTHON',
 'CUDA',
 'NV',
 'QCOM',
 'CPU',
 'NULL',
 'METAL',
 'REMOTE',
 'NPY',
 'AMD',
 'WEBGPU',
 'DISK',
 'HIP',
 'GPU']

In [63]:
%pip install tinygrad


[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [64]:
t = tinygrad.Tensor(np.linspace(0,5,6), dtype='float', device='metal')
c = t * 3
d = c.realize()
d.numpy()

array([ 0.,  3.,  6.,  9., 12., 15.], dtype=float32)

In [65]:

len(set([1,2,3]))

3

In [92]:
def encoder( tokens):
  assert len(set(tokens)) == len(tokens), f'non-unique chars in tokens: {tokens}'
  map = {}
  rmap = {}
  for i,t in enumerate(tokens):
    map[t]=i
    rmap[i] = t
  def encode(data):
    result = np.zeros((len(tokens), len(data)))
    for i, d in enumerate(data):
      result[map[d], i] = 1
    return result
  def decode_one(data):
    assert isinstance(data, np.ndarray)
    for i,d in enumerate(data):
      if d != 0:
        return rmap[i] 
    
  return encode, decode_one

In [ ]:
# what are the dimensions of things?
'''
class RNN():
  def __init__(self, ):
    pass
  def step(self, x):
    # update the hidden state
    self.h = np.tanh(
      np.dot(self.W_hh, self.h) + np.dot(self.W_xh, x)
      )

    # compute the output vec
    y = np.dot(self.W_hy, self.h)
    return y
'''
# in here, we see that the hidden state length 

class RNN():
  def __init__(self, input_size, hidden_size, tokens ):
    self.batch_size = 1
    self.input_size = input_size
    self.hidden_size = hidden_size
    self.encode, self.decode = encoder(tokens)
    self.h = Tensor(np.zeros((self.hidden_size)), dtype='float')
    self.W_xh = Tensor(np.random.uniform(-1, 1, (self.input_size, self.hidden_size) ), dtype='float')
    self.W_hh = Tensor(np.random.uniform(-1, 1, (self.hidden_size, self.hidden_size)) , dtype='float' )
    self.W_hy = Tensor(np.random.uniform(-1, 1, (self.hidden_size, self.input_size) ) , dtype='float') 

  def __repr__(self):
    return f'h:\n{self.h.numpy()}\n----------\nxh:\n{self.W_xh.numpy()}\n----------\nhh:\n{self.W_hh.numpy()}\n----------\nhy:\n{self.W_hy.numpy()}'
  
  def step(self, input):
    input = Tensor(np.asarray(input), dtype='float' )
    p1 = self.W_hh @ self.h.reshape(self.hidden_size, 1).flatten()
    p2 = (input.reshape(1, self.input_size) @ self.W_xh).flatten()
    self.h = (p1 + p2).tanh()
    y =  self.h @ self.W_hy
    return y.numpy()
  
  def forward(self, input):
    enc_input = self.encode(input)
    result = []
    for col in enc_input.T:
      result.append(self.step(col)) 
    
    return ''.join([self.decode(r) for r in result])
  
  def train(self, train):
    pass
  
  def backward(self, input):
    pass

  def predict(self, input):
    pass
  


In [94]:
Tensor(np.array([1,2,3])).reshape(3,1)

<Tensor <UOp METAL (3, 1) long ShapeTracker(views=(View(shape=(3, 1), strides=(1, 0), offset=0, mask=None, contiguous=True),))> on METAL with grad None>

In [95]:
r = RNN(4, 3, 'helo')
y = r.forward('hheelloo')
print(r)




p1 <Tensor <UOp METAL (3,) float ShapeTracker(views=(View(shape=(3,), strides=(1,), offset=0, mask=None, contiguous=True),))> on METAL with grad None> 
p2, <Tensor <UOp METAL (3,) float ShapeTracker(views=(View(shape=(3,), strides=(1,), offset=0, mask=None, contiguous=True),))> on METAL with grad None>
p1 <Tensor <UOp METAL (3,) float ShapeTracker(views=(View(shape=(3,), strides=(1,), offset=0, mask=None, contiguous=True),))> on METAL with grad None> 
p2, <Tensor <UOp METAL (3,) float ShapeTracker(views=(View(shape=(3,), strides=(1,), offset=0, mask=None, contiguous=True),))> on METAL with grad None>
p1 <Tensor <UOp METAL (3,) float ShapeTracker(views=(View(shape=(3,), strides=(1,), offset=0, mask=None, contiguous=True),))> on METAL with grad None> 
p2, <Tensor <UOp METAL (3,) float ShapeTracker(views=(View(shape=(3,), strides=(1,), offset=0, mask=None, contiguous=True),))> on METAL with grad None>
p1 <Tensor <UOp METAL (3,) float ShapeTracker(views=(View(shape=(3,), strides=(1,), offs

In [96]:
y

'hhhhhhhh'